In [53]:
from gensim import corpora, models, similarities
import gensim
from sklearn import svm, metrics
import numpy as np
from gensim.models import doc2vec
from collections import namedtuple
import pandas as pd
from sklearn.ensemble import RandomForestClassifier

In [54]:
#training dataset
df = pd.read_csv('P1_training_set.csv')
e1 = df['Event 1']
e2 = df['Event 2']
label = df['Label']
#print(e1.size*2)

16134


In [55]:
doc1 = e1.tolist()
doc2 = e2.tolist()
doc = doc1+doc2
#print(doc)
#print(len(doc))

In [56]:

#print(doc1)

docs = []
analyzedDocument = namedtuple('AnalyzedDocument', 'words tags')
for i, text in enumerate(doc):
    words = text.lower().split()
    tags = [i]
    docs.append(analyzedDocument(words, tags))
    #print(words,tags)

In [57]:
model = doc2vec.Doc2Vec(docs, size = 100, window = 300, min_count = 1, workers = 4)

C:\Users\spradha5\AppData\Local\Continuum\anaconda3\lib\site-packages\gensim\models\doc2vec.py:574: UserWarning: The parameter `size` is deprecated, will be removed in 4.0.0, use `vector_size` instead.
  warnings.warn("The parameter `size` is deprecated, will be removed in 4.0.0, use `vector_size` instead.")
2019-09-14 14:23:45,957 : INFO : collecting all words and their counts
2019-09-14 14:23:45,958 : INFO : PROGRESS: at example #0, processed 0 words (0/s), 0 word types, 0 tags
2019-09-14 14:23:45,988 : INFO : PROGRESS: at example #10000, processed 55550 words (1908200/s), 6036 word types, 10000 tags
2019-09-14 14:23:46,007 : INFO : collected 7539 word types and 16134 unique tags from a corpus of 16134 examples and 89375 words
2019-09-14 14:23:46,008 : INFO : Loading a fresh vocabulary
2019-09-14 14:23:46,022 : INFO : effective_min_count=1 retains 7539 unique words (100% of original 7539, drops 0)
2019-09-14 14:23:46,023 : INFO : effective_min_count=1 leaves 89375 word corpus (100% o

In [58]:
#training vectors
xtrain = []
ytrain = []

for i in range(e1.size):
    j=i+e1.size
    lab = label.loc[i]
    vec1 = (model.docvecs[i]).tolist()
    vec2 = (model.docvecs[j]).tolist()
    vec = vec1+vec2
    
    xtrain.append(vec)
    ytrain.append(lab)
    
xtrain = np.array(xtrain).astype(np.float32)
ytrain = np.array(ytrain).astype(np.float32)

In [59]:
rfc = RandomForestClassifier(n_jobs=-1, n_estimators=300, max_features='auto')
rfc.fit(xtrain, ytrain)

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
                       max_depth=None, max_features='auto', max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=300,
                       n_jobs=-1, oob_score=False, random_state=None, verbose=0,
                       warm_start=False)

In [60]:
#testing dataset
df2 = pd.read_csv('P1_testing_set.csv')
e1t = df2['Event 1']
e2t = df2['Event 2']
labelt = df2['Label']

In [61]:
doc1t = e1t.tolist()
doc2t = e2t.tolist()
doct = doc1t+doc2t

In [62]:
docst = []
analyzedDocument = namedtuple('AnalyzedDocument', 'words tags')
for i, text in enumerate(doct):
    words = text.lower().split()
    tags = [i]
    docst.append(analyzedDocument(words, tags))

In [63]:
modelt = doc2vec.Doc2Vec(docst, size = 100, window = 300, min_count = 1, workers = 4)

C:\Users\spradha5\AppData\Local\Continuum\anaconda3\lib\site-packages\gensim\models\doc2vec.py:574: UserWarning: The parameter `size` is deprecated, will be removed in 4.0.0, use `vector_size` instead.
  warnings.warn("The parameter `size` is deprecated, will be removed in 4.0.0, use `vector_size` instead.")
2019-09-14 14:24:02,854 : INFO : collecting all words and their counts
2019-09-14 14:24:02,855 : INFO : PROGRESS: at example #0, processed 0 words (0/s), 0 word types, 0 tags
2019-09-14 14:24:02,863 : INFO : collected 2167 word types and 1800 unique tags from a corpus of 1800 examples and 9957 words
2019-09-14 14:24:02,864 : INFO : Loading a fresh vocabulary
2019-09-14 14:24:02,928 : INFO : effective_min_count=1 retains 2167 unique words (100% of original 2167, drops 0)
2019-09-14 14:24:02,929 : INFO : effective_min_count=1 leaves 9957 word corpus (100% of original 9957, drops 0)
2019-09-14 14:24:02,937 : INFO : deleting the raw counts dictionary of 2167 items
2019-09-14 14:24:02,9

In [64]:
#testing vectors
xtest = []
ytest = []

for i in range(e1t.size):
    j=i+e1t.size
    lab = labelt.loc[i]
    vec1 = (modelt.docvecs[i]).tolist()
    vec2 = (modelt.docvecs[j]).tolist()
    vec = vec1+vec2
    
    xtest.append(vec)
    ytest.append(lab)
    
xtest = np.array(xtest).astype(np.float32)
ytest = np.array(ytest).astype(np.float32)

In [65]:
ypred = rfc.predict(xtest)

In [67]:
#Performance Metrics
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report

print('\n Confusion Matrix: \n',confusion_matrix(ytest,ypred))
print('\n Accuracy: ', accuracy_score(ytest,ypred)*100)
print('\n Precision, Recall, F1-score: \n',classification_report(ytest,ypred))


 Confusion Matrix: 
 [[  0 168  19]
 [  0 341  40]
 [  0 304  28]]

 Accuracy:  41.0

 Precision, Recall, F1-score: 
               precision    recall  f1-score   support

         0.0       0.00      0.00      0.00       187
         1.0       0.42      0.90      0.57       381
         2.0       0.32      0.08      0.13       332

    accuracy                           0.41       900
   macro avg       0.25      0.33      0.23       900
weighted avg       0.30      0.41      0.29       900

